**In case of problems or questions, please first check the list of [Frequently Asked Questions (FAQ)](https://stardist.net/docs/faq.html).**

Please shutdown all other training/prediction notebooks before running this notebook (as those might occupy the GPU memory otherwise).

In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist import random_label_cmap
from stardist.models import StarDist3D

np.random.seed(6)
lbl_cmap = random_label_cmap()

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Data

We assume that data has already been downloaded in via notebook [1_data.ipynb](1_data.ipynb).  
We now load images from the sub-folder `test` that have not been used during training.

In [3]:
file_list = sorted(glob(r'E:\RABBIT_DATA\CLAHE+3DMED\*.tif', recursive=True))

In [ ]:
for file in file_list:
    
    X = [file]
    
    cell_name = (file.split('\\')[-1].split('.tiff')[0])
    print("Processing: ", file)
    
    print(X[0], cell_name)
    
    X = list(map(imread,X))

    n_channel = 1 if X[0].ndim == 3 else X[0].shape[-1]
    axis_norm = (0,1,2)   # normalize channels independently
    if n_channel > 1:
        print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
      
    demo_model = False

    if demo_model:
        model = StarDist3D.from_pretrained('3D_demo')
    else:
        model = StarDist3D(None, name='stardist', basedir='models')
    None;


    img = normalize(X[0], 1, 99.8, axis=axis_norm)
    
    print(img.shape)
    
    labels, details = model.predict_instances(img, n_tiles=model._guess_n_tiles(img))

    ## Save predictions
    print("Saving: ", file)
    filename = r'E:\RABBIT_RESULTS\\' + cell_name + '_cluster_labels.tif'
    save_tiff_imagej_compatible(filename, labels, axes='ZYX')